# 主成分分析（PCA）

## 概述

主成分分析（Principal Component Analysis, PCA）是最常用的线性降维方法，通过正交变换将原始特征映射到一组线性不相关的新特征上，这些新特征称为主成分。

## 核心思想

- **几何视角**：在所有 $k$ 维线性子空间中，寻找使投影后方差最大的子空间
- **统计视角**：在均值为0的数据上，PCA寻找协方差矩阵的主特征向量

$$\Sigma = \frac{1}{m} X^\top X, \quad \Sigma u_i = \lambda_i u_i$$

其中 $u_i$ 为第 $i$ 个主成分方向，$\lambda_i$ 为对应的方差（特征值）

## 本节内容

1. PCA 基本使用
2. 解释方差比与成分选择
3. 累计方差可视化
4. 降维效果可视化

## 1. 环境准备

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

# 设置随机种子确保可重复性
np.random.seed(42)

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

## 2. 数据加载与预处理

使用经典的鸢尾花（Iris）数据集进行演示：
- 150个样本，4个特征
- 3个类别：setosa, versicolor, virginica

In [ ]:
# 加载鸢尾花数据集
iris = load_iris()
X, y = iris.data, iris.target

print(f"数据集形状: {X.shape}")
print(f"特征名称: {iris.feature_names}")
print(f"类别名称: {iris.target_names}")

In [ ]:
# 标准化处理（PCA对数据尺度敏感，标准化是必要的预处理步骤）
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"标准化后均值: {X_scaled.mean(axis=0).round(10)}")
print(f"标准化后标准差: {X_scaled.std(axis=0).round(2)}")

## 3. PCA 基本使用

### 3.1 按方差比例选择成分

通过设置 `n_components` 为 (0, 1] 之间的浮点数，可以自动选择保留指定比例方差所需的最少主成分数。

In [ ]:
# 创建 PCA 模型，保留 95% 的方差
pca_95 = PCA(n_components=0.95)
X_pca_95 = pca_95.fit_transform(X_scaled)

print(f"原始维度: {X_scaled.shape[1]}")
print(f"降维后维度: {X_pca_95.shape[1]}")
print(f"保留的成分数: {pca_95.n_components_}")

### 3.2 查看主成分矩阵

`pca.components_` 返回主成分矩阵（$W_d^\top$），每行是一个主成分方向。

In [ ]:
# 查看主成分矩阵
print("主成分矩阵形状:", pca_95.components_.shape)
print("\n主成分矩阵:")
print(pca_95.components_)

# 展示各特征对主成分的贡献
print("\n各特征对主成分的贡献:")
for i, component in enumerate(pca_95.components_):
    print(f"\n主成分 {i+1}:")
    for feature_name, weight in zip(iris.feature_names, component):
        print(f"  {feature_name}: {weight:.4f}")

## 4. 解释方差比

解释方差比（Explained Variance Ratio）表示每个主成分所解释的数据方差占总方差的比例。

In [ ]:
# 首先对完整数据进行 PCA，查看所有成分的方差贡献
pca_full = PCA()
pca_full.fit(X_scaled)

print("各主成分的解释方差比:")
for i, ratio in enumerate(pca_full.explained_variance_ratio_):
    print(f"  主成分 {i+1}: {ratio:.4f} ({ratio*100:.2f}%)")

print(f"\n总解释方差比: {pca_full.explained_variance_ratio_.sum():.4f}")

## 5. 累计方差可视化

累计解释方差曲线用于确定保留多少主成分可以保留足够的信息。

In [ ]:
# 计算累计方差
cumsum = np.cumsum(pca_full.explained_variance_ratio_)

# 找到满足 95% 方差的最小成分数
d_95 = np.argmax(cumsum >= 0.95) + 1

# 绘制累计方差曲线
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左图：单独方差贡献
ax1 = axes[0]
ax1.bar(range(1, len(pca_full.explained_variance_ratio_) + 1), 
        pca_full.explained_variance_ratio_, alpha=0.7, color='steelblue')
ax1.set_xlabel('Principal Component')
ax1.set_ylabel('Explained Variance Ratio')
ax1.set_title('Individual Explained Variance')
ax1.set_xticks(range(1, len(pca_full.explained_variance_ratio_) + 1))

# 右图：累计方差
ax2 = axes[1]
ax2.plot(range(1, len(cumsum) + 1), cumsum, 'o-', color='steelblue', linewidth=2, markersize=8)
ax2.axhline(y=0.95, color='red', linestyle='--', label='95% threshold')
ax2.axvline(x=d_95, color='green', linestyle='--', label=f'{d_95} components')
ax2.fill_between(range(1, d_95 + 1), 0, cumsum[:d_95], alpha=0.3, color='green')
ax2.set_xlabel('Number of Principal Components')
ax2.set_ylabel('Cumulative Explained Variance Ratio')
ax2.set_title('Cumulative Explained Variance')
ax2.set_xticks(range(1, len(cumsum) + 1))
ax2.legend(loc='lower right')
ax2.set_ylim(0, 1.05)

plt.tight_layout()
plt.show()

print(f"保留 95% 方差所需的最少主成分数: {d_95}")

## 6. 降维效果可视化

将4维数据降至2维进行可视化，观察类别分布情况。

In [ ]:
# 降至2维用于可视化
pca_2d = PCA(n_components=2)
X_2d = pca_2d.fit_transform(X_scaled)

# 创建可视化
fig, ax = plt.subplots(figsize=(10, 8))

colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
markers = ['o', 's', '^']

for i, (target_name, color, marker) in enumerate(zip(iris.target_names, colors, markers)):
    mask = y == i
    ax.scatter(X_2d[mask, 0], X_2d[mask, 1], 
               c=color, marker=marker, s=60, alpha=0.7,
               label=target_name, edgecolors='white', linewidth=0.5)

ax.set_xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]*100:.1f}% variance)')
ax.set_ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]*100:.1f}% variance)')
ax.set_title('PCA Projection of Iris Dataset')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"2D投影保留的方差比: {pca_2d.explained_variance_ratio_.sum()*100:.2f}%")

## 7. 双标图（Biplot）

双标图同时展示样本点和特征向量的投影，有助于理解主成分的含义。

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))

# 绘制样本点
for i, (target_name, color, marker) in enumerate(zip(iris.target_names, colors, markers)):
    mask = y == i
    ax.scatter(X_2d[mask, 0], X_2d[mask, 1], 
               c=color, marker=marker, s=50, alpha=0.6,
               label=target_name, edgecolors='white', linewidth=0.5)

# 绘制特征向量
scale = 3  # 缩放因子，便于可视化
for i, (feature_name, component) in enumerate(zip(iris.feature_names, pca_2d.components_.T)):
    ax.arrow(0, 0, component[0] * scale, component[1] * scale,
             head_width=0.1, head_length=0.05, fc='red', ec='red', linewidth=2)
    ax.text(component[0] * scale * 1.15, component[1] * scale * 1.15,
            feature_name, fontsize=10, color='red', ha='center', va='center')

ax.set_xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]*100:.1f}% variance)')
ax.set_ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]*100:.1f}% variance)')
ax.set_title('Biplot: PCA of Iris Dataset')
ax.legend(loc='upper left')
ax.axhline(y=0, color='gray', linestyle='-', linewidth=0.5)
ax.axvline(x=0, color='gray', linestyle='-', linewidth=0.5)
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')

plt.tight_layout()
plt.show()

## 8. 总结

### 关键要点

1. **数据预处理**：PCA对数据尺度敏感，通常需要先进行标准化

2. **成分数选择**：
   - 指定具体数值：`PCA(n_components=2)`
   - 指定方差比例：`PCA(n_components=0.95)`
   - 通过累计方差曲线分析

3. **重要属性**：
   - `components_`：主成分矩阵（投影方向）
   - `explained_variance_ratio_`：各成分解释方差比
   - `n_components_`：实际使用的成分数

4. **应用场景**：
   - 数据可视化（降至2D/3D）
   - 特征压缩与去噪
   - 加速后续模型训练
   - 消除多重共线性

### 注意事项

- PCA是线性方法，对于非线性结构可能效果有限
- 主成分是原始特征的线性组合，可解释性降低
- 对异常值敏感，考虑使用稳健PCA变体